# Opioid
***
Model based off the prescription opioid crisis in the United States. Based off a paper from the University of Tennessee https://0afa17f2-bd49-4985-b62b-358fb4a6bf3f.filesusr.com/ugd/f70b03_22c7703e4a3b4da6b9555c738ed8566d.pdf
***
## Setup the Environment
***

In [1]:
import copy
import hashlib

In [2]:
import numpy

MatPlotLib and Plotly are used for creating custom visualizations

In [3]:
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [4]:
from plotly.offline import iplot
import plotly.graph_objs as go

In [5]:
import gillespy2

In [6]:
import time
import csv

In [7]:
from distributed import Client
from distributed.diagnostics.progressbar import progress

In [8]:
import os
from dask.distributed import Client, LocalCluster

os.environ['MALLOC_TRIM_THRESHOLD_'] = '8192'

cluster = LocalCluster(n_workers=4, nanny=True)
client = Client(cluster)


/home/ubuntu/.local/lib/python3.8/site-packages/distributed/dashboard/core.py:20: UserWarning:


Dask needs bokeh >= 2.4.2, < 3 for the dashboard.
You have bokeh==3.0.3.
Continuing without the dashboard.

2023-04-27 12:01:12,561 - distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2023-04-27 12:01:12,567 - distributed.scheduler - INFO - State start
2023-04-27 12:01:12,573 - distributed.scheduler - INFO -   Scheduler at:     tcp://127.0.0.1:40471
2023-04-27 12:01:12,574 - distributed.scheduler - INFO -   dashboard at:            127.0.0.1:8787
2023-04-27 12:01:12,636 - distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:44709'
2023-04-27 12:01:12,650 - distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:46517'
2023-04-27 12:01:12,666 - distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:46481'
2023-04-27 12:01:12,672 - distributed.nanny 

***
## Create the Opioid Model
***

In [9]:
def create_opioid(t):
    model = gillespy2.Model(name='Opioid')
    model.volume = 1

    # Variables
    Susceptible = gillespy2.Species(name='Susceptible', initial_value=1000, mode='discrete')
    Prescribed_User = gillespy2.Species(name='Prescribed_User', initial_value=0, mode='discrete')
    Addicted = gillespy2.Species(name='Addicted', initial_value=0, mode='discrete')
    Rehab = gillespy2.Species(name='Rehab', initial_value=0, mode='discrete')
    Natural_Deaths = gillespy2.Species(name='Natural_Deaths', initial_value=0, mode='discrete')
    Addicted_Deaths = gillespy2.Species(name='Addicted_Deaths', initial_value=0, mode='discrete')
    model.add_species([
        Susceptible, Prescribed_User, Addicted, Rehab, Natural_Deaths, Addicted_Deaths
    ])

    # Parameters
    alpha = gillespy2.Parameter(name='alpha', expression='0.15')
    epsilon = gillespy2.Parameter(name='epsilon', expression='0.8')
    beta_p = gillespy2.Parameter(name='beta_p', expression='0.00266')
    beta_a = gillespy2.Parameter(name='beta_a', expression='0.00094')
    gamma = gillespy2.Parameter(name='gamma', expression='0.00744')
    zeta = gillespy2.Parameter(name='zeta', expression='0.2')
    delta = gillespy2.Parameter(name='delta', expression='0.1')
    sigma = gillespy2.Parameter(name='sigma', expression='0.9')
    mu = gillespy2.Parameter(name='mu', expression='0.00729')
    mu_prime = gillespy2.Parameter(name='mu_prime', expression='0.01159')
    model.add_parameter([
        alpha, epsilon, beta_p, beta_a, gamma, zeta, delta, sigma, mu, mu_prime
    ])

    # Reactions
    SP = gillespy2.Reaction(
        name='SP', rate='alpha',
        reactants={'Susceptible': 1}, products={'Prescribed_User': 1}
    )
    SA_a = gillespy2.Reaction(
        name='SA_a', rate='beta_a',
        reactants={'Susceptible': 1}, products={'Addicted': 1}
    )
    SA_p = gillespy2.Reaction(
        name='SA_p', rate='beta_p',
        reactants={'Susceptible': 1}, products={'Addicted': 1}
    )
    PA = gillespy2.Reaction(
        name='PA', rate='gamma',
        reactants={'Prescribed_User': 1}, products={'Addicted': 1}
    )
    PS = gillespy2.Reaction(
        name='PS', rate='epsilon',
        reactants={'Prescribed_User': 1}, products={'Susceptible': 1}
    )
    AR = gillespy2.Reaction(
        name='AR', rate='zeta',
        reactants={'Addicted': 1}, products={'Rehab': 1}
    )
    RA = gillespy2.Reaction(
        name='RA', rate='delta',
        reactants={'Rehab': 1}, products={'Addicted': 1}
    )
    RS = gillespy2.Reaction(
        name='RS', rate='sigma',
        reactants={'Rehab': 1}, products={'Susceptible': 1}
    )
    mu_S = gillespy2.Reaction(
        name='mu_S', rate='mu',
        reactants={'Susceptible': 1}, products={'Susceptible': 1, 'Natural_Deaths': 1}
    )
    mu_P = gillespy2.Reaction(
        name='mu_P', rate='mu',
        reactants={'Prescribed_User': 1}, products={'Susceptible': 1, 'Natural_Deaths': 1}
    )
    mu_R = gillespy2.Reaction(
        name='mu_R', rate='mu',
        reactants={'Rehab': 1}, products={'Susceptible': 1, 'Natural_Deaths': 1}
    )
    mu_prime_A = gillespy2.Reaction(
        name='mu_prime_A', rate='mu_prime',
        reactants={'Addicted': 1}, products={'Susceptible': 1, 'Addicted_Deaths': 1}
    )
    model.add_reaction([
        SP, SA_a, SA_p, PA, PS, AR, RA, RS, mu_S, mu_P, mu_R, mu_prime_A
    ])

    # Timespan
    #tspan = gillespy2.TimeSpan.arange(1, t=200)
    tspan = gillespy2.TimeSpan.arange(1, t)
    model.timespan(tspan)
    return model

***
## Simulation Parameters
***

In [11]:
def configure_simulation(model, n):
    solver = gillespy2.SSACSolver(model=model, delete_directory=False)
    kwargs = {
        'number_of_trajectories': n,
        'timeout':20,
        # 'seed': None,
        # 'tau_tol': 0.03,
        # 'integrator_options': {'rtol': 0.001, 'atol': 1e-06},
        'solver': solver,
    }
    return kwargs

***
## Parameter Sweep
***

In [12]:
class ParameterSweep:
    def __init__(self, t, kwargs, model):
        self.tspan = t
        self.model = model
        self.settings = {}
        self.parameters = []
        self.__parameters = {}
        self.kwargs = kwargs

        self.results = {}
        self.__results = {}

    def __reset(self):
        self.results = {}
        self.__results = {}
        
    def __write_to_file(self, filename):
        with open(filename, 'a', encoding='UTF8', newline = "") as f:
            writer = csv.writer(f)
            row = []
            row.append(self.runtime)
            paras = ['alpha', 'epsilon', 'beta_p', 'beta_a', 'gamma', 'zeta', 'delta', 'sigma', 'mu', 'mu_prime']
            #paras = ['alpha', 'epsilon']
            for para in paras:
                row.append(self.settings["variables"][para])
            row.append(self.settings["number_of_trajectories"])
            row.append(self.tspan)
            writer.writerow(row)
  
    def __run(self, variables, index, verbose=False):
        if index < len(self.parameters):
            parameter = self.parameters[index]
            index += 1
            for value in parameter['range']:
                variables[parameter['param']] = value
                self.__run(variables, index, verbose=verbose)
        else:
            start = time.time()
            sim_key = hashlib.md5(str(variables).encode('utf-8')).hexdigest()
            if sim_key not in self.results:
                model = self.__setup_model(variables=variables)
                if verbose:
                    print(f'--> running: {str(variables)}')
                self.results[sim_key] = model.run(**self.settings)
            end = time.time()
            self.runtime = end - start
            self.__write_to_file("opioid_data.csv")
            #print(self.settings["variables"])
            #print(self.settings)

    def __resume(self, resume):
        orig_params = {}
        self.results = resume.data
        for parameter in resume.parameters:
            orig_params[parameter['param']] = parameter['range']
            self.add_parameter(parameter['param'], values= parameter['range'])
        new_params = [param for param in self.__parameters if param not in orig_params]
        if len(new_params) > 0:
            variables = {new_param: self.model.listOfParameters[new_param].value for new_param in new_params}
            self.__update_results(resume.parameters, variables, {})
            self.results = self.__results

    def __setup_model(self, variables):
        if 'solver' in self.settings and 'CSolver' in self.settings['solver'].name:
            self.settings['variables'] = copy.deepcopy(variables)
            return self.model
        model = copy.deepcopy(self.model)
        for name, value in variables.items():
            model.listOfParameters[name].expression = str(value)
        return model

    def __update_results(self, parameters, variables, orig_vars, index=0):
        if index < len(parameters):
            parameter = parameters[index]
            index += 1
            for value in parameter['range']:
                orig_vars[parameter['param']] = value
                variables[parameter['param']] = value
                self.__update_results(parameters, variables, orig_vars, index=index)
        else:
            o_sim_key = hashlib.md5(str(orig_vars).encode('utf-8')).hexdigest()
            n_sim_key = hashlib.md5(str(variables).encode('utf-8')).hexdigest()
            self.__results[n_sim_key] = self.results[o_sim_key]

    def add_parameter(self, param, bounds=None, steps=11, values=None):
        if values is None:
            values = numpy.linspace(bounds[0], bounds[1], steps)
        else:
            values = numpy.array(values)
        if param in self.__parameters:
            for value in values:
                if value not in self.__parameters[param]:
                    self.__parameters[param] = numpy.append(self.__parameters[param], value)
            self.__parameters[param] = numpy.sort(self.__parameters[param])
        else:
            self.__parameters[param] = values
        self.parameters = [{'param': param, 'range': ps_range} for param, ps_range in self.__parameters.items()]

    def run(self, resume=None, verbose=False):
        self.settings = self.kwargs

        if resume is not None:
            self.__resume(resume)
        index = 0
        variables = {}
        self.timeout = self.settings["timeout"]
        self.__run(variables, index, verbose=verbose)

        results = Results(self.results, self.parameters)
        self.__reset()
        return results

In [13]:
class Results:
    func_map = {
        'min': numpy.min, 'max': numpy.max, 'avg': numpy.mean,
        'var': numpy.var, 'final': lambda res: res[-1]
    }

    def __init__(self, data, parameters):
        self.data = data
        self.visible = False
        self.parameters = parameters

    def feature_extractor(self, species, results=None, key='final', verbose=False):
        if results is None:
            results = self.data.values()
        data = [[self.func_map[key](traj[species]) for traj in result] for result in results]
        if verbose:
            print(f'{key} populations for {species}: {data}')
        return numpy.array(data)

    def ensemble_aggregator(self, results, key='avg', verbose=False):
        data = [self.func_map[key](result) for result in results]
        if verbose:
            print(f'{key} of the ensembles: {data}')
        return numpy.array(data)
    
    def plot(self, species, fe_key='final', ea_key='avg', verbose=False):
        x_data = self.parameters[0]['range']
        y_data = self.parameters[1]['range']
        fe_data = self.feature_extractor(species, key=fe_key, verbose=verbose)
        ea_data = self.ensemble_aggregator(fe_data, key=ea_key, verbose=verbose)
        shape = (len(self.parameters[1]['range']), len(self.parameters[0]['range']))
        data = numpy.flip(numpy.reshape(ea_data, shape))

        fig, ax = plt.subplots(figsize=(16, 2 * len(data)))
        plt.imshow(data)
        ax.set_xticks(numpy.arange(data.shape[1]) + 0.5, minor=False)
        ax.set_yticks(numpy.arange(data.shape[0]) + 0.5, minor=False)
        plt.title(f'Parameter Sweep - Variable: {species}', fontsize=18, fontweight='bold')
        ax.set_xticklabels(x_data)
        ax.set_yticklabels(y_data)
        ax.set_xlabel(self.parameters[1]['param'], fontsize=16, fontweight='bold')
        ax.set_ylabel(self.parameters[0]['param'], fontsize=16, fontweight='bold')
        ax.tick_params(axis='x', labelsize=14, labelrotation=90)
        ax.tick_params(axis='y', labelsize=14)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.2)
        _ = plt.colorbar(ax=ax, cax=cax)

    def plotplotly(self, species, fe_key='final', ea_key='avg',
                        return_plotly_figure=False, verbose=False):
        x_data = self.parameters[1]['range']
        y_data = self.parameters[0]['range']
        fe_data = self.feature_extractor(species, key=fe_key, verbose=verbose)
        ea_data = self.ensemble_aggregator(fe_data, key=ea_key, verbose=verbose)
        shape = (len(self.parameters[1]['range']), len(self.parameters[0]['range']))
        data = numpy.reshape(ea_data, shape)

        trace_list = [go.Heatmap(z=data, x=x_data, y=y_data)]

        title = f'<b>Parameter Sweep - Variable: {species}</b>'
        layout = go.Layout(
            title=dict(text=title, x=0.5),
            xaxis=dict(title=f'<b>{self.parameters[1]["param"]}</b>'),
            yaxis=dict(title=f'<b>{self.parameters[0]["param"]}</b>')
        )

        fig = dict(data=trace_list, layout=layout)
        if return_plotly_figure:
            return fig
        iplot(fig)

In [14]:
def run(psweep):
    return psweep.run()

In [15]:
with open("opioid_data.csv", 'w', encoding='UTF8', newline = "") as f:
        writer = csv.writer(f)
        row = ["runtime", 'alpha', 'epsilon', 'beta_p', 'beta_a', 'gamma', 'zeta', 'delta', 'sigma', 'mu', 'mu_prime', "number_of_trajectories", "Timespan"]
        writer.writerow(row)

In [16]:
paras = ['alpha', 'epsilon', 'beta_p', 'beta_a', 'gamma', 'zeta', 'delta', 'sigma', 'mu', 'mu_prime']
ps = []
for t in range(100,201,100):
    model = create_opioid(t)
    for k in (1,3):
        kwargs = configure_simulation(model, k)
        psweep = ParameterSweep(t, kwargs, model)
        for i, para in enumerate(paras):
            p_bounds = (
            0.9 * float(eval(model.get_parameter(para).expression)),
            1.1 * float(eval(model.get_parameter(para).expression))
            )
            psweep.add_parameter(para, bounds=p_bounds, steps=3)

        ps.append(psweep)

In [17]:
futures = client.map(run, psweep)

In [19]:
progress(futures)

VBox()